In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Logical Question Answering in English and Indic using LLM-
based Models.
compare how the models are performing
see how transformer models understand and resolve logic-based questions,

choosen types of inferenctial reasoning : Blood reation, direction based and cause-effect
Options ; Syllogism, direction based, blood realtion, number series, puzzle based, cause-effect or assumption based MCQ

Dataset used : AI4Bharat Logical Reasoning Dataset, IndicQA
Options : SQuAD v2, IndicQA Dataset, AI4Bharat Logical Reasoning Dataset

Model selection : 
general LLM(English) - FlanT5
Indic language (hindi) - IndicBert
Multilingual - XLM-R

Evaluation matrix - 
Exact match (EM) - Measures how many answers exactly match the ground truth.
F1 Score - Considers partial matches (important if model predicts slightly different wording but logically correct answer).

Custom score for logical reasoning
Model confidence

In [ ]:
from huggingface_hub import login
login(token="your_huggingface_token")

In [ ]:
from datasets import load_dataset
dataset = load_dataset("ai4bharat/MILU", data_dir="English")

In [ ]:
!pip uninstall -y tensorflow tensorflow-intel tensorflow-gpu
import os
os.environ["USE_TF"] = "0"
os.environ["USE_FLAX"] = "0"

import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


In [1]:
!pip install --upgrade transformers -q
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 106.4 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 31.9 MB/s eta 0:00:00


In [6]:
# ---------- Upgrade & Environment Fix ----------
!pip install --upgrade transformers datasets scikit-learn -q
!pip uninstall -y tensorflow tensorflow-intel tensorflow-gpu -q

import os
os.environ["USE_TF"] = "0"
os.environ["USE_FLAX"] = "0"

# ---------- Imports ----------
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ---------- Load Dataset (LogiQA English MCQ) ----------
dataset = load_dataset("lucasmccabe/logiqa", split="test")
dataset = dataset.select(range(min(100, len(dataset))))  # For quick test

# ---------- Load Flan-T5 ----------
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

# ---------- Answer Generation ----------
def generate_answer(prompt_text):
    inputs = tokenizer(prompt_text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=16, num_beams=4)
    return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

# ---------- Prediction Loop ----------
predictions, gold_labels, questions, options_list, raw_outputs = [], [], [], [], []

for ex in dataset:
    context = ex["context"]
    question = ex["query"]
    opts = ex["options"]
    correct = ex["correct_option"]

    # Prompt for Flan‑T5
    prompt = f"Context: {context}\nQuestion: {question}\nOptions:\n"
    for idx, opt in enumerate(opts):
        prompt += f"{chr(65+idx)}. {opt}\n"
    prompt += "Answer:"

    answer = generate_answer(prompt)
    answer_upper = answer.upper().strip()
    pred_label = next((chr(65+i) for i in range(len(opts)) if answer_upper.startswith(chr(65+i))), "A")

    predictions.append(pred_label)
    gold_labels.append(chr(65 + correct))
    questions.append(question)
    options_list.append(", ".join(opts))
    raw_outputs.append(answer)



# ---------- Metrics ----------
accuracy = accuracy_score(gold_labels, predictions)
precision, recall, f1, _ = precision_recall_fscore_support(gold_labels, predictions, average="weighted")

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# ---------- Save to CSV ----------
df = pd.DataFrame({
    "Question": questions,
    "Options": options_list,
    "TrueAnswer": gold_labels,
    "PredictedLabel": predictions,
    "RawAnswer": raw_outputs
})
df.to_csv("flan_t5_logiqa_predictions.csv", index=False)
print("Saved predictions to flan_t5_logiqa_predictions.csv")


Using the latest cached version of the dataset since lucasmccabe/logiqa couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /root/.cache/huggingface/datasets/lucasmccabe___logiqa/default/0.0.0/3c19b0488d794d30c36f73d132d8a22e64f42f2e (last modified on Thu Jul 31 07:00:41 2025).


Accuracy: 0.2800
Precision: 0.3137
Recall: 0.2800
F1 Score: 0.2904
Saved predictions to flan_t5_logiqa_predictions.csv


In [5]:
print(dataset.column_names)
print(dataset[0])


['context', 'query', 'options', 'correct_option']
{'context': 'In the planning of a new district in a township, it was decided to build a special community in the southeast, northwest, centered on the citizen park. These four communities are designated as cultural area, leisure area, commercial area and administrative service area. It is known that the administrative service area is southwest of the cultural area, and the cultural area is southeast of the leisure area.', 'query': 'Based on the above statement, which of the following can be derived?', 'options': ['Civic Park is north of the administrative service area.', 'The leisure area is southwest of the cultural area.', 'The cultural district is in the northeast of the business district.', 'The business district is southeast of the leisure area.'], 'correct_option': 0}


In [9]:
# ---------------- Install Required ----------------
!pip install transformers accelerate -q

# ---------------- Imports ----------------
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import shutil

# ---------------- Load Falcon ----------------
model_name = "tiiuae/falcon-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

# ---------------- Inference Function ----------------
def generate_falcon_answer(context, question, options):
    prompt = f"""
You are a reasoning assistant. Choose the best option only as A, B, C, or D.

Context: {context}
Question: {question}
Options:
A. {options[0]}
B. {options[1]}
C. {options[2]}
D. {options[3]}

Answer with only A, B, C, or D.
"""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=20)
    raw_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    for ch in ["A", "B", "C", "D"]:
        if ch in raw_answer:
            return ch, raw_answer
    return "A", raw_answer  # fallback

# ---------------- Dataset (Already Loaded for Flan‑T5) ----------------
predictions, gold_labels, questions, options_list, raw_outputs = [], [], [], [], []

for ex in dataset:  # dataset you already loaded for flan‑t5
    context = ex["context"]
    question = ex["query"]
    opts = ex["options"]
    correct = ex["correct_option"]

    pred, raw = generate_falcon_answer(context, question, opts)

    predictions.append(pred)
    gold_labels.append(chr(65 + correct))
    questions.append(question)
    options_list.append(", ".join(opts))
    raw_outputs.append(raw)

# ---------------- Metrics ----------------
accuracy = accuracy_score(gold_labels, predictions)
precision, recall, f1, _ = precision_recall_fscore_support(gold_labels, predictions, average="weighted")

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# ---------------- Save Predictions ----------------
df = pd.DataFrame({
    "Question": questions,
    "Options": options_list,
    "TrueAnswer": gold_labels,
    "PredictedLabel": predictions,
    "RawAnswer": raw_outputs
})
df.to_csv("falcon7b_logiqa_predictions.csv", index=False)
print("Predictions saved to falcon7b_logiqa_predictions.csv")

# ---------------- Save Model Locally for Frontend ----------------
shutil.make_archive("falcon_7b_instruct", 'zip', model_name)
print("Model archived as falcon_7b_instruct.zip")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_tok

Accuracy: 0.2200
Precision: 0.0484
Recall: 0.2200
F1 Score: 0.0793
Predictions saved to falcon7b_logiqa_predictions.csv


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  y_true : 1d array-like, or label indicator array / sparse matrix


FileNotFoundError: [Errno 2] No such file or directory: 'tiiuae/falcon-7b-instruct'

Deepseek

In [14]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd
import gc

# ---- Device ----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ---- English Model (Flan-T5-Large) ----
english_model_name = "google/flan-t5-base"
english_tokenizer = AutoTokenizer.from_pretrained(english_model_name)
english_model = AutoModelForSeq2SeqLM.from_pretrained(
    english_model_name,
    torch_dtype=torch.float16
).to(device)

# ---- Metrics ----
def exact_match(pred, gold):
    return int(pred.strip().lower() == gold.strip().lower())

def compute_f1(pred, gold):
    pred_tokens, gold_tokens = pred.split(), gold.split()
    common = set(pred_tokens) & set(gold_tokens)
    if not common:
        return 0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(gold_tokens)
    return 2 * (precision * recall) / (precision + recall)

def model_confidence(logits):
    probs = torch.nn.functional.softmax(logits, dim=-1)
    max_probs = probs.max(dim=-1).values
    return max_probs.mean().item()

# ---- Evaluate ----
def evaluate_flan_t5(dataset):
    results = []
    for _, row in dataset.iterrows():
        question, answer = row["question"], row["answer"]

        inputs = english_tokenizer(question, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = english_model.generate(
                **inputs, 
                max_length=64,
                output_scores=True,
                return_dict_in_generate=True
            )
        pred = english_tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
        logits = torch.stack(outputs.scores, dim=1).squeeze(0)
        confidence = model_confidence(logits)

        em = exact_match(pred, answer)
        f1 = compute_f1(pred, answer)
        adjusted_conf = confidence if em == 1 else confidence * 0.5

        results.append({
            "question": question,
            "answer": answer,
            "prediction": pred,
            "EM": em,
            "F1": f1,
            "Confidence": adjusted_conf
        })
    return results

# ---- Example dataset ----
dataset = pd.DataFrame({
    "question": [
        "A is father of B. Who is B to A?",
        "X moves North, then East. Which direction is X from starting point?"
    ],
    "answer": ["Son", "North-East"]
})

# ---- Run Flan-T5-Large ----
res_english = evaluate_flan_t5(dataset)
print("\n--- English Model (Flan-T5-Large) ---")
for r in res_english:
    print(r)

# ---- Clean GPU memory ----
del english_model
torch.cuda.empty_cache()
gc.collect()


Using device: cuda

--- English Model (Flan-T5-Large) ---
{'question': 'A is father of B. Who is B to A?', 'answer': 'Son', 'prediction': 'a', 'EM': 0, 'F1': 0, 'Confidence': 0.08894210308790207}
{'question': 'X moves North, then East. Which direction is X from starting point?', 'answer': 'North-East', 'prediction': 'east', 'EM': 0, 'F1': 0, 'Confidence': 0.2898373305797577}


156

In [15]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd
import gc

# ---- Device ----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ---- English Model (Flan-T5-Base for safety) ----
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16
).to(device)

# ---- Metrics ----
def exact_match(pred, gold):
    return int(pred.strip().lower() == gold.strip().lower())

def compute_f1(pred, gold):
    pred_tokens, gold_tokens = pred.split(), gold.split()
    common = set(pred_tokens) & set(gold_tokens)
    if not common:
        return 0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(gold_tokens)
    return 2 * (precision * recall) / (precision + recall)

def model_confidence(logits):
    probs = torch.nn.functional.softmax(logits, dim=-1)
    max_probs = probs.max(dim=-1).values
    return max_probs.mean().item()

# ---- Evaluate ----
def evaluate_model(dataset, base_prompt="", few_shot=False):
    results = []
    few_shot_examples = """You are an expert in logical reasoning.
Examples:
Q: A is father of B. Who is B to A?
A: Son
Q: X moves North, then East. Which direction is X from starting point?
A: North-East
"""

    for _, row in dataset.iterrows():
        question, answer = row["question"], row["answer"]

        if few_shot:
            prompt = f"{few_shot_examples}\nNow answer:\nQ: {question}\nA:"
        else:
            prompt = f"{base_prompt}\nQuestion: {question}\nAnswer:"

        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model.generate(
                **inputs, 
                max_length=64,
                output_scores=True,
                return_dict_in_generate=True
            )
        pred = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
        logits = torch.stack(outputs.scores, dim=1).squeeze(0)
        confidence = model_confidence(logits)

        em = exact_match(pred, answer)
        f1 = compute_f1(pred, answer)
        adjusted_conf = confidence if em == 1 else confidence * 0.5

        results.append({
            "question": question,
            "answer": answer,
            "prediction": pred,
            "EM": em,
            "F1": f1,
            "Confidence": adjusted_conf
        })
    return results

# ---- Aggregated Evaluation ----
def evaluation_matrix(results):
    em = sum(r["EM"] for r in results) / len(results)
    f1 = sum(r["F1"] for r in results) / len(results)
    conf = sum(r["Confidence"] for r in results) / len(results)
    return {"Exact Match": em, "F1 Score": f1, "Confidence": conf}

# ---- Example dataset ----
dataset = pd.DataFrame({
    "question": [
        "A is father of B. Who is B to A?",
        "X moves North, then East. Which direction is X from starting point?"
    ],
    "answer": ["Son", "North-East"]
})

# ---- User input for custom prompt ----
print("\nEnter your custom base prompt (press Enter for default):")
user_prompt = input().strip()
if not user_prompt:
    user_prompt = "You are an expert in logical reasoning. Think step by step and give the correct answer."

print("Do you want to use few-shot examples? (y/n):")
use_few_shot = input().strip().lower() == "y"

# ---- Run evaluation ----
results = evaluate_model(dataset, base_prompt=user_prompt, few_shot=use_few_shot)

# ---- Print results ----
print("\n--- Detailed Predictions ---")
for r in results:
    print(r)

print("\n--- Evaluation Matrix ---")
matrix = evaluation_matrix(results)
print(matrix)

# ---- Clean up GPU memory ----
del model
torch.cuda.empty_cache()
gc.collect()


Using device: cuda

Enter your custom base prompt (press Enter for default):


 if ram is the father of sam, then what is sam to ram


Do you want to use few-shot examples? (y/n):


 y



--- Detailed Predictions ---
{'question': 'A is father of B. Who is B to A?', 'answer': 'Son', 'prediction': 'Son', 'EM': 1, 'F1': 1.0, 'Confidence': 0.8456429243087769}
{'question': 'X moves North, then East. Which direction is X from starting point?', 'answer': 'North-East', 'prediction': 'North-East', 'EM': 1, 'F1': 1.0, 'Confidence': 0.9072210788726807}

--- Evaluation Matrix ---
{'Exact Match': 1.0, 'F1 Score': 1.0, 'Confidence': 0.8764320015907288}


156

In [17]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd
import gc

# ---- Device ----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ---- Model ----
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16
).to(device)

# ---- Metrics ----
def exact_match(pred, gold):
    return int(pred.strip().lower() == gold.strip().lower())

def compute_f1(pred, gold):
    pred_tokens, gold_tokens = pred.split(), gold.split()
    common = set(pred_tokens) & set(gold_tokens)
    if not common:
        return 0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(gold_tokens)
    return 2 * (precision * recall) / (precision + recall)

def model_confidence(logits):
    probs = torch.nn.functional.softmax(logits, dim=-1)
    max_probs = probs.max(dim=-1).values
    return max_probs.mean().item()

# ---- Evaluate dataset ----
def evaluate_model(dataset, base_prompt="", few_shot=False):
    few_shot_examples = """You are an expert in logical reasoning.
Examples:
Q: A is father of B. Who is B to A?
A: Son
Q: X moves North, then East. Which direction is X from starting point?
A: North-East
"""

    results = []
    for _, row in dataset.iterrows():
        question, answer = row["question"], row["answer"]

        prompt = (f"{few_shot_examples}\nNow answer:\nQ: {question}\nA:"
                  if few_shot else
                  f"{base_prompt}\nQuestion: {question}\nAnswer:")

        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_length=64,
                output_scores=True,
                return_dict_in_generate=True
            )
        pred = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
        logits = torch.stack(outputs.scores, dim=1).squeeze(0)
        confidence = model_confidence(logits)

        em = exact_match(pred, answer)
        f1 = compute_f1(pred, answer)
        adjusted_conf = confidence if em == 1 else confidence * 0.5

        results.append({
            "question": question,
            "answer": answer,
            "prediction": pred,
            "EM": em,
            "F1": f1,
            "Confidence": adjusted_conf
        })
    return results

def evaluation_matrix(results):
    em = sum(r["EM"] for r in results) / len(results)
    f1 = sum(r["F1"] for r in results) / len(results)
    conf = sum(r["Confidence"] for r in results) / len(results)
    return {"Exact Match": em, "F1 Score": f1, "Confidence": conf}

# ---- Dataset ----
dataset = pd.DataFrame({
    "question": [
        "A is father of B. Who is B to A?",
        "X moves North, then East. Which direction is X from starting point?"
    ],
    "answer": ["Son", "North-East"]
})

# ---- User Prompt ----
print("\nEnter your custom base prompt (press Enter for default):")
user_prompt = input().strip()
if not user_prompt:
    user_prompt = "You are an expert in logical reasoning. Think step by step and give the correct answer."

print("Do you want to use few-shot examples? (y/n):")
use_few_shot = input().strip().lower() == "y"

# ---- Evaluate dataset ----
results = evaluate_model(dataset, base_prompt=user_prompt, few_shot=use_few_shot)
print("\n--- Dataset Predictions ---")
for r in results:
    print(r)
print("\n--- Evaluation Matrix ---")
print(evaluation_matrix(results))

# ---- Interactive Question Mode ----
while True:
    user_question = input("\nEnter your own logical question (or type 'exit'): ")
    if user_question.lower() == "exit":
        break
    prompt = (f"You are an expert in logical reasoning.\n{user_prompt}\n"
              f"Question: {user_question}\nAnswer:")
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=64)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"Answer: {answer}")

# ---- Cleanup ----
del model
torch.cuda.empty_cache()
gc.collect()


Using device: cuda

Enter your custom base prompt (press Enter for default):


 if neha is the mother of sneha then what is sneha to neha


Do you want to use few-shot examples? (y/n):


 y



--- Dataset Predictions ---
{'question': 'A is father of B. Who is B to A?', 'answer': 'Son', 'prediction': 'Son', 'EM': 1, 'F1': 1.0, 'Confidence': 0.8456429243087769}
{'question': 'X moves North, then East. Which direction is X from starting point?', 'answer': 'North-East', 'prediction': 'North-East', 'EM': 1, 'F1': 1.0, 'Confidence': 0.9072210788726807}

--- Evaluation Matrix ---
{'Exact Match': 1.0, 'F1 Score': 1.0, 'Confidence': 0.8764320015907288}


...

Answer: neha is the mother of sneha



Enter your own logical question (or type 'exit'):  Rahul starts from his house and walks 4 km towards the North, then turns right and walks 3 km. In which direction is he from his starting point?


Answer: North



Enter your own logical question (or type 'exit'):  exit


246

In [21]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

save_path = "/kaggle/working/flan-t5-base"
tokenizer.save_pretrained(save_path)
model.save_pretrained(save_path)


In [22]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

save_path = "/kaggle/working/flan-t5-base"
tokenizer.save_pretrained(save_path)
model.save_pretrained(save_path)


In [23]:
import shutil
shutil.make_archive("/kaggle/working/flan-t5-base", 'zip', "/kaggle/working/flan-t5-base")


'/kaggle/working/flan-t5-base.zip'

In [24]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

save_path = "/kaggle/working/flan-t5-base"
tokenizer.save_pretrained(save_path)
model.save_pretrained(save_path)
import shutil
shutil.make_archive("/kaggle/working/flan-t5-base", 'zip', save_path)



'/kaggle/working/flan-t5-base.zip'

Indiclanguage

In [2]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# IndicBERT QA fine-tuned model (if you have one)
model_name = "ai4bharat/indic-bert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

save_path = "/kaggle/working/indic-bert"
tokenizer.save_pretrained(save_path)
model.save_pretrained(save_path)


config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/5.65M [00:00<?, ?B/s]

2025-08-05 00:19:07.655173: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754353148.023618      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754353148.125924      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


pytorch_model.bin:   0%|          | 0.00/135M [00:00<?, ?B/s]

Some weights of AlbertForQuestionAnswering were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
import shutil
shutil.make_archive("/kaggle/working/indic-bert", 'zip', save_path)


model.safetensors:   0%|          | 0.00/135M [00:00<?, ?B/s]

'/kaggle/working/indic-bert.zip'

In [7]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

model_name = "deepset/xlm-roberta-base-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name).to("cpu")

context = "राम सीता के पति हैं।"
question = "सीता के पति कौन हैं?"

inputs = tokenizer(question, context, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)

start_idx = torch.argmax(outputs.start_logits)
end_idx = torch.argmax(outputs.end_logits)

# Handle case where model predicts no valid answer
if start_idx == 0 and end_idx == 0:
    answer = "No answer found"
else:
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_idx:end_idx+1])
    # Remove special tokens
    tokens = [t for t in tokens if not t.startswith("<")]
    answer = tokenizer.convert_tokens_to_string(tokens)

print("Answer:", answer)




Answer: No answer found


In [9]:
!pip install transformers sentencepiece

from transformers import MarianTokenizer, MarianMTModel, AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Load translation models once
def load_translation_models():
    hi_en_name = "Helsinki-NLP/opus-mt-hi-en"
    en_hi_name = "Helsinki-NLP/opus-mt-en-hi"
    hi_en_tokenizer = MarianTokenizer.from_pretrained(hi_en_name)
    hi_en_model = MarianMTModel.from_pretrained(hi_en_name)
    en_hi_tokenizer = MarianTokenizer.from_pretrained(en_hi_name)
    en_hi_model = MarianMTModel.from_pretrained(en_hi_name)
    return (hi_en_tokenizer, hi_en_model, en_hi_tokenizer, en_hi_model)

hi_en_tokenizer, hi_en_model, en_hi_tokenizer, en_hi_model = load_translation_models()

def translate_hi_to_en(text):
    tokens = hi_en_tokenizer(text, return_tensors="pt", truncation=True)
    translated = hi_en_model.generate(**tokens)
    return hi_en_tokenizer.decode(translated[0], skip_special_tokens=True)

def translate_en_to_hi(text):
    tokens = en_hi_tokenizer(text, return_tensors="pt", truncation=True)
    translated = en_hi_model.generate(**tokens)
    return en_hi_tokenizer.decode(translated[0], skip_special_tokens=True)

# --- Example usage ---
hindi_text = "राम सीता के पति हैं।"
english_text = translate_hi_to_en(hindi_text)
print("Hindi → English:", english_text)

hindi_back = translate_en_to_hi("Ram is Sita's husband.")
print("English → Hindi:", hindi_back)



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/813k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/306M [00:00<?, ?B/s]

Hindi → English: Ram Sita's husband.
English → Hindi: राम सीता के पति है.


In [11]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# Load Hindi QA model
def load_hindi_qa_model():
    model_name = "deepset/xlm-roberta-base-squad2"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForQuestionAnswering.from_pretrained(model_name).to("cpu")
    return tokenizer, model

hindi_tokenizer, hindi_model = load_hindi_qa_model()

def get_hindi_answer(question, context, threshold=0.2):
    inputs = hindi_tokenizer(question, context, return_tensors="pt")
    with torch.no_grad():
        outputs = hindi_model(**inputs)

    # Probabilities
    start_probs = F.softmax(outputs.start_logits, dim=-1)
    end_probs = F.softmax(outputs.end_logits, dim=-1)
    start_idx = torch.argmax(outputs.start_logits)
    end_idx = torch.argmax(outputs.end_logits)
    confidence = (start_probs[0, start_idx] * end_probs[0, end_idx]).item()

    if confidence < threshold or (start_idx == 0 and end_idx == 0):
        return "No answer found (low confidence)", confidence

    tokens = hindi_tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_idx:end_idx+1])
    tokens = [t for t in tokens if not t.startswith("<")]
    answer = hindi_tokenizer.convert_tokens_to_string(tokens)
    return answer, confidence


In [12]:
answer, confidence = get_hindi_answer(question, context)
if "No answer found" in answer:
    # Translate question & context to English
    question_en = translate_hi_to_en(question)
    context_en = translate_hi_to_en(context)

    # Run English QA model (Flan-T5 or any generative model)
    inputs = flan_tokenizer(question_en, context_en, return_tensors="pt")
    with torch.no_grad():
        outputs = flan_model.generate(**inputs)
    english_answer = flan_tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Translate back to Hindi
    answer_hi = translate_en_to_hi(english_answer)
    st.write(f"**Fallback Answer (via translation):** {answer_hi}")


NameError: name 'flan_tokenizer' is not defined

In [14]:
# --- Load Hindi QA model (from earlier) ---
hindi_tokenizer, hindi_model = load_hindi_qa_model()

# --- Load English QA model ---
flan_model_name = "google/flan-t5-base"
flan_tokenizer = AutoTokenizer.from_pretrained(flan_model_name)
flan_model = AutoModelForSeq2SeqLM.from_pretrained(flan_model_name).to("cpu")

# --- Hindi Question & Context ---
question = "सीता के पति कौन हैं?"
context = "राम सीता के पति हैं।"

# --- Run Hindi QA ---
answer, confidence = get_hindi_answer(question, context)

if "No answer found" in answer:
    # Translate Hindi → English
    question_en = translate_hi_to_en(question)
    context_en = translate_hi_to_en(context)

    # Run English QA (Flan-T5)
    inputs = flan_tokenizer(question_en + " Context: " + context_en, return_tensors="pt")
    with torch.no_grad():
        outputs = flan_model.generate(**inputs)
    english_answer = flan_tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Translate English answer → Hindi
    answer = translate_en_to_hi(english_answer)

print("Final Answer:", answer)
print("Confidence (Hindi model):", confidence)



tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Final Answer: राम सीता
Confidence (Hindi model): 0.8960077166557312


In [15]:
def normalize_text(text):
    import re
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9\u0900-\u097F\s]", "", text)  # keep Hindi + English
    return " ".join(text.split())

def compute_f1(pred, truth):
    pred_tokens = normalize_text(pred).split()
    truth_tokens = normalize_text(truth).split()
    common = set(pred_tokens) & set(truth_tokens)
    if len(common) == 0:
        return 0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(truth_tokens)
    return 2 * precision * recall / (precision + recall)

def evaluate_dataset(dataset):
    results = []
    total_em, total_f1, total_conf = 0, 0, 0

    for data in dataset:
        question = data["question"]
        context = data["context"]
        ground_truth = data["answer"]

        # Hindi QA
        answer, confidence = get_hindi_answer(question, context)
        if "No answer found" in answer:
            # Fallback (English QA)
            question_en = translate_hi_to_en(question)
            context_en = translate_hi_to_en(context)
            inputs = flan_tokenizer(question_en + " Context: " + context_en, return_tensors="pt")
            with torch.no_grad():
                outputs = flan_model.generate(**inputs)
            english_answer = flan_tokenizer.decode(outputs[0], skip_special_tokens=True)
            answer = translate_en_to_hi(english_answer)

        # Metrics
        em = 1 if normalize_text(answer) == normalize_text(ground_truth) else 0
        f1 = compute_f1(answer, ground_truth)

        total_em += em
        total_f1 += f1
        total_conf += confidence

        results.append({
            "question": question,
            "answer": ground_truth,
            "prediction": answer,
            "EM": em,
            "F1": f1,
            "Confidence": confidence
        })

    n = len(dataset)
    matrix = {
        "Exact Match": total_em / n,
        "F1 Score": total_f1 / n,
        "Avg Confidence": total_conf / n
    }
    return results, matrix


In [16]:
# Example Hindi dataset
dataset = [
    {"question": "सीता के पति कौन हैं?", "context": "राम सीता के पति हैं।", "answer": "राम"},
    {"question": "भारत की राजधानी क्या है?", "context": "भारत की राजधानी नई दिल्ली है।", "answer": "नई दिल्ली"}
]

results, matrix = evaluate_dataset(dataset)
print("--- Detailed Predictions ---")
for r in results:
    print(r)
print("\n--- Evaluation Matrix ---")
print(matrix)


--- Detailed Predictions ---
{'question': 'सीता के पति कौन हैं?', 'answer': 'राम', 'prediction': 'राम सीता', 'EM': 0, 'F1': 0.6666666666666666, 'Confidence': 0.8960077166557312}
{'question': 'भारत की राजधानी क्या है?', 'answer': 'नई दिल्ली', 'prediction': 'नई दिल्ली', 'EM': 1, 'F1': 1.0, 'Confidence': 0.8035038113594055}

--- Evaluation Matrix ---
{'Exact Match': 0.5, 'F1 Score': 0.8333333333333333, 'Avg Confidence': 0.8497557640075684}


In [1]:
def evaluate_dataset_with_accuracy(dataset):
    total_em, total_f1, total_conf, correct, results = 0, 0, 0, 0, []
    for d in dataset:
        ans, conf = get_indicbert_answer(d["question"], d["context"])
        em = 1 if normalize_text(ans) == normalize_text(d["answer"]) else 0
        f1 = compute_f1(ans, d["answer"])
        total_em += em
        total_f1 += f1
        total_conf += conf
        if em == 1:  # Count correct answers
            correct += 1
        results.append({
            "question": d["question"],
            "answer": d["answer"],
            "prediction": ans,
            "EM": em,
            "F1": f1,
            "Confidence": conf
        })
    
    n = len(dataset)
    metrics = {
        "Exact Match": total_em/n,
        "F1 Score": total_f1/n,
        "Avg Confidence": total_conf/n,
        "Accuracy": correct/n  # <--- Added Accuracy
    }
    return results, metrics
